In [2]:
import pandas as pd
import xml.etree.ElementTree as et
import os
import re
import urllib.request
import keras.backend
import keras.models
import matplotlib.pyplot as plt
from matplotlib import cm, transforms
import numpy as np
import pickle
import time

Using TensorFlow backend.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([(

In [3]:
xtree = et.parse(urllib.request.urlopen("https://raw.githubusercontent.com/nicklogin/NLPTeam3000/master/development/SentiRuEval_rest_train.xml"))

In [4]:
#!wget https://github.com/nicklogin/NLPTeam3000/archive/master.zip

In [5]:
#!unzip master.zip.2

In [6]:
root = xtree.getroot()

In [7]:
data = []
columns = ['id', 'food', 'service', 'text']

In [8]:
for review in root:
    text_id = int(review.attrib['id'])
    
    scores = review.find('scores')
    
    food = int(scores.find('food').text)
    service = int(scores.find('service').text)
    
    text = review.find('text').text

    new_food = 0
    new_service = 0

    if food > 5:
        new_food = 1

    if service > 5:
        new_service = 1
    
    data.append({'id': text_id,
                'food': new_food,
                'service': new_service,
                'text': text})
    

df = pd.DataFrame(data, columns=columns)
df = df.set_index('id')

In [9]:
root[0].find('text').text

'И пускай на меня не обижается наш прославленный защитник - франкофон «Монреаль Канадиенс» Maxime – я всегда с некоторой опаской относился к этому народу. Народу способному с таким благоговением доводить до цирроза всю пернатую живность, заставлять специальных поисковых свиней копошиться в грязи в поисках сумчатых грибов, ковыряться в тине, собирая брюхоногих и двустворчатых. Народу, подсадившего все население земного шара на муть, именуемое «Божоле Нуво», на сыры с плесенью и на квакающих жаб, скачущих по болотам. Тем не менее, единожды заглянув, не мог успокоиться, пока не вкусил большую часть меню. Не буду вдаваться в детали «вкусно-невкусно», а приведу турнирную таблицу, коли начали с хоккея. 1.Петух, запеченный в сливках, с картофелем и грибами. Почему петух, а не курица? Различать еще не научился. 2.Домашний пирог. Яблочный. Горячий. Тающий. 3.Говядина по-бургунски. Кускус неожиданный. 4.Террин из печени цыпленка. Массивная порция. 5.Утиная ножка «Конфи» с картофельным пюре. Мягк

In [10]:
test_txts = []
food_negs = []
food_poss = []
service_negs = []
service_poss = []



for file in os.listdir("Downloads/NLPTeam3000-master-y/conllu_data"):
    if file.endswith('.tsv'):
        file_text = open(os.path.join("Downloads/NLPTeam3000-master-y/conllu_data", file), 'r', encoding='utf-8').read()
        text = ''
        sentences = []
        for l in file_text.splitlines():
            if '# text = ' in l:
                sentences.append(l.replace('# text = ', ''))
                text += l.replace('# text = ', '')
    test_txts.append(text)


    food_neg = []
    food_pos = []
    service_neg = []
    service_pos = []


    markup = open(os.path.join('Downloads/NLPTeam3000-master-y/разметка_финал', file[:-4]+".tsv"), 'r', encoding='utf-8').read()
    for line in markup.splitlines():
        line = line.strip()
        if line:
            l = line.split('\t')[0]
            ws = []
            for n in line.split('\t')[1].split(','):
                ws.append(sentences[int(l) - 1][int(n) - 1])

            if line.split('\t')[2] == 'Food' and int(line.split('\t')[3]) == 0:
                food_neg.append(ws)
            elif line.split('\t')[2] == 'Food' and int(line.split('\t')[3]) == 1:
                food_pos.append(ws)
            elif line.split('\t')[2] == 'Service' and int(line.split('\t')[3]) == 0:
                service_neg.append(ws)
            elif line.split('\t')[2] == 'Service' and int(line.split('\t')[3]) == 1:
                service_pos.append(ws)

    food_negs.append(food_neg)
    food_poss.append(food_pos)
    service_negs.append(service_neg)
    service_poss.append(service_pos)

In [11]:
print(len(test_txts))
print(len(food_negs))
print(len(food_poss))
print(len(service_negs))
print(len(service_poss))

10
10
10
10
10


In [12]:
texts = []
foods = []
services = []

for review in root:
    
    scores = review.find('scores')
    
    food = int(scores.find('food').text)
    service = int(scores.find('service').text)
    
    texts.append(review.find('text').text)

    new_food = 0
    new_service = 0

    if food > 5:
        new_food = 1

    if service > 5:
        new_service = 1
    
    foods.append(new_food)
    services.append(new_service)

In [13]:
print(len(texts))
print(len(foods))
print(len(services))

19034
19034
19034


In [14]:
#!wget http://vectors.nlpl.eu/repository/11/180.zip
#!unzip 180.zip

In [15]:
def max_length(texts):
    return max(len(t) for t in texts)

In [16]:
from gensim.models import KeyedVectors
w2v_modelf = KeyedVectors.load_word2vec_format('model.bin', binary=True)
w2v_models = KeyedVectors.load_word2vec_format('model.bin', binary=True)

In [17]:
from sklearn.model_selection import train_test_split
scores_trainf, scores_valf, texts_trainf, texts_valf = train_test_split(
    foods[:18943], texts[:18943], test_size=0.3)

In [18]:
from sklearn.model_selection import train_test_split
scores_trains, scores_vals, texts_trains, texts_vals = train_test_split(
    services[:18943], texts[:18943], test_size=0.3)

In [19]:
from collections import Counter
from itertools import chain

MAX_LENf = max(max_length(texts_trainf), max_length(texts_valf))
MAX_LENs = max(max_length(texts_trains), max_length(texts_vals))

In [20]:
def load_datasetf(lines, embedding_dim, num_examples=None):
    prep = lines[:num_examples]
    vocab = Counter()
    x_tensor = np.zeros((len(prep), MAX_LENf, embedding_dim))
    for i, text in enumerate(prep):
        for j, w in enumerate(text):
            try:
                x_tensor[i, j, :] = w2v_modelf[w]
            except KeyError:
                pass
        vocab[w] += 1
    return x_tensor, vocab

In [21]:
def load_datasets(lines, embedding_dim, num_examples=None):
    prep = lines[:num_examples]
    vocab = Counter()
    x_tensor = np.zeros((len(prep), MAX_LENs, embedding_dim))
    for i, text in enumerate(prep):
        for j, w in enumerate(text):
            try:
                x_tensor[i, j, :] = w2v_models[w]
            except KeyError:
                pass
        vocab[w] += 1
    return x_tensor, vocab

In [22]:
input_tensor_trainf, inp_vocab_trainf = load_datasetf(texts_trainf, w2v_modelf.vector_size)
input_tensor_valf, inp_vocab_valf = load_datasetf(texts_valf, w2v_modelf.vector_size)

In [23]:
input_tensor_trains, inp_vocab_trains = load_datasets(texts_trains, w2v_models.vector_size)
input_tensor_vals, inp_vocab_vals = load_datasets(texts_vals, w2v_models.vector_size)

In [24]:
w2v_modelf.vector_size

300

In [25]:
w2v_models.vector_size

300

In [26]:
input_tensor_trainf.shape

(13260, 23438, 300)

In [27]:
input_tensor_trains.shape

(13260, 23438, 300)

In [30]:
embedding_dimf = w2v_modelf.vector_size
inp_vocabf = inp_vocab_trainf + inp_vocab_valf
vocab_inp_sizef = len(inp_vocabf)+1

In [31]:
embedding_dims = w2v_models.vector_size
inp_vocabs = inp_vocab_trains + inp_vocab_vals
vocab_inp_sizes = len(inp_vocabs)+1

In [32]:
from innvestigate.utils.tests.networks import base as network_base
def build_network(max_len, voc_size, embedding_dim, output_n, activation=None, dense_unit=256, dropout_rate=0.25):
    if activation:
        activation = "relu"

    net = {}
    net["in"] = keras.Input(shape=[1, max_len, embedding_dim])
    net["conv"] = keras.layers.Conv2D(filters=100, kernel_size=(1,2), strides=(1, 1), padding='valid')(net["in"])
    net["pool"] = keras.layers.MaxPooling2D(pool_size=(1, max_len-1), strides=(1,1))(net["conv"])
    net["out"] = network_base.dense_layer(keras.layers.Flatten()(net["pool"]), units=output_n, activation=activation)
    net["sm_out"] = network_base.softmax(net["out"])


    net.update({
        "input_shape": [1, max_len, embedding_dim],
        "output_n": output_n,
    })
    return net

In [33]:
netf = build_network(MAX_LENf, vocab_inp_sizef, embedding_dimf, 2)
model_without_softmaxf = keras.models.Model(inputs=netf['in'], outputs=netf['out'])
model_with_softmaxf = keras.models.Model(inputs=netf['in'], outputs=netf['sm_out'])

W1224 23:55:32.361495 4642790848 deprecation_wrapper.py:119] From /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1224 23:55:32.389413 4642790848 deprecation_wrapper.py:119] From /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1224 23:55:32.397359 4642790848 deprecation_wrapper.py:119] From /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1224 23:55:32.423907 4642790848 deprecation_wrapper.py:119] From /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:39

In [34]:
nets = build_network(MAX_LENs, vocab_inp_sizes, embedding_dims, 2)
model_without_softmaxs = keras.models.Model(inputs=nets['in'], outputs=nets['out'])
model_with_softmaxs = keras.models.Model(inputs=nets['in'], outputs=nets['sm_out'])

In [35]:
model_without_softmaxs.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 1, 23438, 300)     0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 1, 23437, 100)     60100     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 1, 1, 100)         0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 202       
Total params: 60,302
Trainable params: 60,302
Non-trainable params: 0
_________________________________________________________________


In [36]:
model_without_softmaxf.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1, 23438, 300)     0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 1, 23437, 100)     60100     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 1, 1, 100)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 202       
Total params: 60,302
Trainable params: 60,302
Non-trainable params: 0
_________________________________________________________________


In [37]:
def to_one_hot(y):
    return keras.utils.to_categorical(y, 2)

def train_modelf(model, epochs=20):
    
    x_train = np.expand_dims(input_tensor_trainf, axis=1)
    y_train = to_one_hot(scores_trainf)
    
    x_val = np.expand_dims(input_tensor_valf, axis=1)
    y_val = to_one_hot(scores_valf)

    model.compile(loss='categorical_crossentropy',
                  optimizer=keras.optimizers.Adam(),
                  metrics=['accuracy'])

    history = model.fit(x_train, y_train,
                        batch_size=25,
                        epochs=epochs,
                        verbose=1,
                        validation_data=(x_val, y_val),
                        shuffle=True
                       )

def train_models(model, epochs=20):
    
    x_train = np.expand_dims(input_tensor_trains, axis=1)
    y_train = to_one_hot(scores_trains)
    
    x_val = np.expand_dims(input_tensor_vals, axis=1)
    y_val = to_one_hot(scores_vals)

    model.compile(loss='categorical_crossentropy',
                  optimizer=keras.optimizers.Adam(),
                  metrics=['accuracy'])

    history = model.fit(x_train, y_train,
                        batch_size=25,
                        epochs=epochs,
                        verbose=1,
                        validation_data=(x_val, y_val),
                        shuffle=True
                       )

In [ ]:
train_modelf(model_with_softmaxf, epochs=3)

W1224 23:55:32.640120 4642790848 deprecation_wrapper.py:119] From /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W1224 23:55:32.655418 4642790848 deprecation_wrapper.py:119] From /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.

W1224 23:55:32.803020 4642790848 deprecation.py:323] From /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W1224 23:55:32.869693 4642790848 deprecation_wrapper.py:119] From /Library/Frameworks

Train on 13260 samples, validate on 5683 samples
Epoch 1/3
 1100/13260 [=>............................] - ETA: 52:01 - loss: 0.6137 - acc: 0.7400

In [ ]:
train_models(model_with_softmaxs, epochs=3)

In [ ]:
model_without_softmaxf.set_weights(model_with_softmaxf.get_weights())

In [ ]:
model_without_softmaxs.set_weights(model_with_softmaxs.get_weights())

In [ ]:
methods = ['gradient', 'lrp.z', 'lrp.alpha_2_beta_1', 'pattern.attribution']
kwargs = [{}, {}, {}, {'pattern_type': 'relu'}]

In [ ]:
import investigate
analyzersf = []

for method, kws in zip(methods, kwargs):
    analyzerf = innvestigate.create_analyzer(method, model_without_softmaxf, **kws)
    analyzerf.fit(np.expand_dims(input_tensor_trainf, axis=1), batch_size=256, verbose=1)
    analyzersf.append(analyzerf)
    
analyzerss = []

for method, kws in zip(methods, kwargs):
    analyzers = innvestigate.create_analyzer(method, model_without_softmaxs, **kws)
    analyzers.fit(np.expand_dims(input_tensor_trains, axis=1), batch_size=256, verbose=1)
    analyzerss.append(analyzers)

In [ ]:
def analyze_scoresf(X, Y, ridx):
    max_len = max_length(input_tensor_trainf)

    analysis = np.zeros([len(analyzers), 1, max_len])
    x, y = X[ridx], Y[ridx]
    t_start = time.time()
    x = x.reshape((1, 1, max_len, embedding_dimf))
    presm = model_without_softmaxf.predict_on_batch(x)[0] #forward pass without softmax
    prob = model_with_softmaxf.predict_on_batch(x)[0] #forward pass with softmax
    y_hat = prob.argmax()
  
    for aidx, analyzer in enumerate(analyzersf):
        a = np.squeeze(analyzer.analyze(x))
        a = np.sum(a, axis=1)
        analysis[aidx] = a
    t_elapsed = time.time() - t_start
    print('Review %d (%.4fs)'% (ridx, t_elapsed))
    return analysis, y_hat

In [ ]:
def analyze_scoress(X, Y, ridx):
    max_len = max_length(input_tensor_trains)

    analysis = np.zeros([len(analyzers), 1, max_len])
    x, y = X[ridx], Y[ridx]
    t_start = time.time()
    x = x.reshape((1, 1, max_len, embedding_dims))
    presm = model_without_softmaxs.predict_on_batch(x)[0] #forward pass without softmax
    prob = model_with_softmaxs.predict_on_batch(x)[0] #forward pass with softmax
    y_hat = prob.argmax()
  
    for aidx, analyzer in enumerate(analyzerss):
        a = np.squeeze(analyzer.analyze(x))
        a = np.sum(a, axis=1)
        analysis[aidx] = a
    t_elapsed = time.time() - t_start
    print('Review %d (%.4fs)'% (ridx, t_elapsed))
    return analysis, y_hat

In [ ]:
analyze_scoresf(input_tensor_trainf, scores_trainf, 97)

In [ ]:
analyze_scoress(input_tensor_trains, scores_trains, 97)

In [ ]:
def plot_text_heatmap(words, scores, title="", width=5, height=0.2, verbose=0, max_word_per_line=10):
    fig = plt.figure(figsize=(width, height))
    
    ax = plt.gca()

    ax.set_title(title, loc='left')
    tokens = words
    if verbose > 0:
        print('len words : %d | len scores : %d' % (len(words), len(scores)))

    cmap = plt.cm.ScalarMappable(cmap=cm.bwr)
    cmap.set_clim(0, 1)
    
    canvas = ax.figure.canvas
    t = ax.transData

    # normalize scores to the followings:
    # - negative scores in [0, 0.5]
    # - positive scores in (0.5, 1]
    normalized_scores = 0.5 * scores / np.max(np.abs(scores)) + 0.5
    
    if verbose > 1:
        print('Raw score')
        print(scores)
        print('Normalized score')
        print(normalized_scores)

    # make sure the heatmap doesn't overlap with the title
    loc_y = -0.2

    for i, token in enumerate(tokens):
        *rgb, _ = cmap.to_rgba(normalized_scores[i], bytes=True)
        color = '#%02x%02x%02x' % tuple(rgb)
        
        text = ax.text(0.0, loc_y, token,
                       bbox={
                           'facecolor': color,
                           'pad': 5.0,
                           'linewidth': 1,
                           'boxstyle': 'round,pad=0.5'
                       }, transform=t)

        text.draw(canvas.get_renderer())
        ex = text.get_window_extent()
        
        # create a new line if the line exceeds the length
        if (i+1) % max_word_per_line == 0:
            loc_y = loc_y -  2.5
            t = ax.transData
        else:
            t = transforms.offset_copy(text._transform, x=ex.width+15, units='dots')

    if verbose == 0:
        ax.axis('off')def plot_text_heatmap(words, scores, title="", width=5, height=0.2, verbose=0, max_word_per_line=10):
    fig = plt.figure(figsize=(width, height))
    
    ax = plt.gca()

    ax.set_title(title, loc='left')
    tokens = words
    if verbose > 0:
        print('len words : %d | len scores : %d' % (len(words), len(scores)))

    cmap = plt.cm.ScalarMappable(cmap=cm.bwr)
    cmap.set_clim(0, 1)
    
    canvas = ax.figure.canvas
    t = ax.transData

    # normalize scores to the followings:
    # - negative scores in [0, 0.5]
    # - positive scores in (0.5, 1]
    normalized_scores = 0.5 * scores / np.max(np.abs(scores)) + 0.5
    
    if verbose > 1:
        print('Raw score')
        print(scores)
        print('Normalized score')
        print(normalized_scores)

    # make sure the heatmap doesn't overlap with the title
    loc_y = -0.2

    for i, token in enumerate(tokens):
        *rgb, _ = cmap.to_rgba(normalized_scores[i], bytes=True)
        color = '#%02x%02x%02x' % tuple(rgb)
        
        text = ax.text(0.0, loc_y, token,
                       bbox={
                           'facecolor': color,
                           'pad': 5.0,
                           'linewidth': 1,
                           'boxstyle': 'round,pad=0.5'
                       }, transform=t)

        text.draw(canvas.get_renderer())
        ex = text.get_window_extent()
        
        # create a new line if the line exceeds the length
        if (i+1) % max_word_per_line == 0:
            loc_y = loc_y -  2.5
            t = ax.transData
        else:
            t = transforms.offset_copy(text._transform, x=ex.width+15, units='dots')

    if verbose == 0:
        ax.axis('off')

In [ ]:
af, y_predf = analyze_scoresf(input_tensor_trainf, scores_trainf, 100)

In [ ]:
ass, y_preds = analyze_scoress(input_tensor_trains, scores_trains, 100)

In [1]:
af[0][0]

NameError: name 'af' is not defined

In [ ]:
ass[0][0]

In [ ]:
plot_text_heatmap(
    texts_train[100],
    af[0][0]
)

In [ ]:
plot_text_heatmap(
    texts_train[100],
    ass[0][0]
)

In [ ]:
idx = 0
wordsf = texts_valf[idx]
wordss = texts_vals[idx]
    
print('Review(id=%d): %s' % (idx, ' '.join(wordsf)))
y_truef = scores_valf[idx]
af, y_predf = analyze_scoresf(input_tensor_valf, scores_valf, idx)

print("Pred class : %d %s" %
      (y_predf, '✓' if y_predf == y_truef else '✗ (%d)' % y_truef)
      )
                            
for j, method in enumerate(methods):
    plot_text_heatmap(wordsf, af[j].reshape(-1), title='Method: %s' % method, verbose=0)
    plt.show()
    print()
    
print('Review(id=%d): %s' % (idx, ' '.join(wordss)))
y_trues = scores_vals[idx]
ass, y_preds = analyze_scoress(input_tensor_vals, scores_vals, idx)

print("Pred class : %d %s" %
      (y_preds, '✓' if y_preds == y_trues else '✗ (%d)' % y_trues)
      )

for j, method in enumerate(methods):
    plot_text_heatmap(wordss, ass[j].reshape(-1), title='Method: %s' % method, verbose=0)
    plt.show()
    print()